<a href="https://colab.research.google.com/github/SnehaVPujari007/Crypto-Data-Analysis/blob/main/Crypto_Currency_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython import get_ipython
from IPython.display import display
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import time

scope = ["https://spreadsheets.google.com/feeds",
         "https://www.googleapis.com/auth/drive",
         'https://www.googleapis.com/auth/spreadsheets']
creds = ServiceAccountCredentials.from_json_keyfile_name("/content/melodic-eye-425616-e1-7cce7250f0a4.json", scope)
client = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1oSlCyyUgCVWJ14caOcYVMfL2ZFLot0x3yoJveeVVvRI/edit?usp=sharing"

spreadsheet_key = "1oSlCyyUgCVWJ14caOcYVMfL2ZFLot0x3yoJveeVVvRI"
sheet = client.open_by_key(spreadsheet_key).sheet1

# Fetch Cryptocurrency Data
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False,
    }
    response = requests.get(url, params=params)
    return response.json()

# Update Data in Google Sheets
def update_sheet():
    # Fetch data
    data = fetch_crypto_data()

    # Clear existing sheet data
    sheet.clear()

    # Add headers
    headers = ["Name", "Symbol", "Price (USD)", "Market Cap", "Volume (24h)", "Price Change (24h %)"]
    sheet.append_row(headers)

    # Add cryptocurrency data
    for coin in data:
        row = [
            coin["name"],
            coin["symbol"],
            coin["current_price"],
            coin["market_cap"],
            coin["total_volume"],
            coin["price_change_percentage_24h"],
        ]
        sheet.append_row(row)

    # Calculate additional values
    highest_change = max(data, key=lambda x: x["price_change_percentage_24h"])
    lowest_change = min(data, key=lambda x: x["price_change_percentage_24h"])
    top_5 = sorted(data, key=lambda x: x["market_cap"], reverse=True)[:5]
    top_5_df = pd.DataFrame(top_5)
    avg_price = sum([coin["current_price"] for coin in data]) / len(data)

    # Add calculated values to the sheet
    sheet.append_row(["", "", "", "", "", ""])  # Add an empty row for spacing
    sheet.append_row(["Highest Price Change:", highest_change["name"], highest_change["price_change_percentage_24h"]])
    sheet.append_row(["Lowest Price Change:", lowest_change["name"], lowest_change["price_change_percentage_24h"]])
    sheet.append_row(["", "", "", "", "", ""])  # Add an empty row for spacing
    sheet.append_row(["Average Price:", avg_price])
    sheet.append_row(["", "", "", "", "", ""])  # Add an empty row for spacing
    sheet.append_row(["Top 5 Cryptocurrencies by Market Cap:"])
    # Add top_5_df to the sheet
    for index, row in top_5_df.iterrows():
        sheet.append_row([row["name"], row["symbol"], row["current_price"], row["market_cap"], row["total_volume"], row["price_change_percentage_24h"]])

# Run Updates Periodically
while True:
    update_sheet()
    print("Google Sheet updated.")
    time.sleep(300)  # Update every 5 minutes # Increased sleep time to 30

Google Sheet updated.
